### Setup

In [1]:
import numpy as np

from hot_dust import preprocess, model

In [2]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [3]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [4]:
epochs = 10000
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
network = model.compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 normalization (Normalizatio  (None, 11)               23        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                384       
                                                                 
 dense_3 (Dense)             (None, 8)                 264       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 680
Trainable params: 657
Non-trainable params: 23
______________________________________________________________

### Fitting

In [6]:
fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
)
network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/10000
126/126 - 1s - loss: 9.9565 - val_loss: 8.8344 - 737ms/epoch - 6ms/step
Epoch 2/10000
126/126 - 0s - loss: 6.9950 - val_loss: 5.9209 - 108ms/epoch - 857us/step
Epoch 3/10000
126/126 - 0s - loss: 5.1508 - val_loss: 4.7618 - 118ms/epoch - 933us/step
Epoch 4/10000
126/126 - 0s - loss: 4.5403 - val_loss: 4.3607 - 124ms/epoch - 982us/step
Epoch 5/10000
126/126 - 0s - loss: 4.2065 - val_loss: 4.0657 - 106ms/epoch - 838us/step
Epoch 6/10000
126/126 - 0s - loss: 3.9023 - val_loss: 3.7710 - 106ms/epoch - 837us/step
Epoch 7/10000
126/126 - 0s - loss: 3.5965 - val_loss: 3.4793 - 107ms/epoch - 849us/step
Epoch 8/10000
126/126 - 0s - loss: 3.2855 - val_loss: 3.1824 - 118ms/epoch - 939us/step
Epoch 9/10000
126/126 - 0s - loss: 2.9915 - val_loss: 2.9167 - 106ms/epoch - 842us/step
Epoch 10/10000
126/126 - 0s - loss: 2.7420 - val_loss: 2.7022 - 105ms/epoch - 831us/step
Epoch 11/10000
126/126 - 0s - loss: 2.5494 - val_loss: 2.5395 - 182ms/epoch - 1ms/step
Epoch 12/10000
126/126 - 0s - loss

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [7]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)